# Kapitel 5: mehrschichtige neuronale Netze

## Multi-Layer-Perceptron

In [4]:
import numpy as np

class MLP(object):
    # die identische Funktion
    def func_id(self, x):
        return x
    
    # weltberühmte Aktivierungsfunktion: Sigmoide
    def func_sigmoid(self, x):
        # np.exp statt math.exp wegen array-operationen
        return 1.0 / (1.0 + np.exp(-x))
    
    # die heutzutage beliebtere Rectifier-Funktion (für schnelleres Lernen) -> Neuron ReLU (Rectified Linear Unit)
    def func_relu(self, x):
        return np.maximum(x, 0)
    
    def __init__(self, n_input_neurons=2, n_hidden_neurons=2, n_output_neurons=1, weights=None):
        """Initialisierung des Netzwerks
        
        Wir verwenden eine fixe I-H-O-Struktur für den Anfang, Neuronenanzahl flexibel
        Initialisierung mit Gewichten möglich [W_IH, W_HO]
        """
        # Anzahl Neuronen pro Layer
        self.n_input_neurons = n_input_neurons
        self.n_hidden_neurons = n_hidden_neurons
        self.n_output_neurons = n_output_neurons
        # Gewichte
        self.weights = weights
        W_IH = []
        W_HO = []
        
        # Daten zur Netzberechnung
        self.network = []
        
        # Input Layer + Bias : Spalten = o_i
        self.inputLayer = np.zeros((self.n_input_neurons + 1, 1))
        self.inputLayer[0] = 1.0
        self.network.append(self.inputLayer)
        
        # Gewichte zum Hidden Layer
        if weights:
            W_IH = self.weights[0]
        else:
            W_IH = np.zeros((self.n_hidden_neurons + 1, self.n_input_neurons + 1))
        self.network.append(W_IH)
        
        # Hidden Layer + Bias : Spalten = net_i, a_i, o_i
        self.hiddenLayer = np.zeros((self.n_hidden_neurons + 1, 3))
        self.hiddenLayer[0] = 1.0
        self.network.append(self.hiddenLayer)
        
        # Gewichte zum Output
        if weights:
            W_HO = self.weights[1]
        else:
            W_HO = np.zeros((self.n_output_neurons + 1, self.n_hidden_neurons + 1))
        self.network.append(W_HO)
        
        # Output Layer + Bias : Spalten = net_i, a_i, o_i (Bias 0, da nicht relevant)
        self.outputLayer = np.zeros((self.n_output_neurons + 1, 3))
        self.outputLayer[0] = 0.0
        self.network.append(self.outputLayer)
    
    def print(self):
        print('Multi-Layer-Perceptron - Netzwerkarchitektur')
        np.set_printoptions(\
            formatter={'float': lambda x: "{0:0.3f}".format(x)})
        for idx, nn_part in enumerate(self.network):
            print(nn_part)
            print('-----------v-----------')
    
    def predict(self, x):
        """Für Eingabe x wird Ausgabe y_hat berechnet
        
        Vorhersage für Vektor x
        """
        # Input Layer setzen: alle Zeilen, Spalte 0
        self.network[0][:,0] = x
        
        # Hidden Layer (start bei 1 wegen Bias-Neuron)
        # net_i
        self.network[2][1:,0] = np.dot(self.network[1][1:,:], self.network[0][:,0])
        # a_i
        self.network[2][1:,1] = self.func_sigmoid(self.network[2][1:,0])
        # o_i
        self.network[2][1:,2] = self.func_id(self.network[2][1:,1])
        
        # Output Layer
        # net_i
        self.network[4][1:,0] = np.dot(self.network[3][1:,:], self.network[2][:,2])
        # a_i
        self.network[4][1:,1] = self.func_sigmoid(self.network[4][1:,0])
        # o_i mit round
        self.network[4][1:,2] = np.round(self.func_id(self.network[4][1:,1]))
        
        return self.network[4][1:,2]
    
#######################################################################################

# Initialisierung
W_IH = np.matrix([[0,0,0],[-10,20,20],[30,-20,-20]])
W_HO = np.matrix([[0,0,0],[-30,20,20]])
weights = []
weights.append(W_IH)
weights.append(W_HO)

nn = MLP(weights=weights)
nn.print()

# Test
X = np.array([[1,1,1],[1,0,1],[1,1,0],[1,0,0]])
y = np.array([0,1,1,0])

print('Predict:')
for idx, x in enumerate(X):
    print('{} {} -> {}'.format(x, y[idx], nn.predict(x)))

Multi-Layer-Perceptron - Netzwerkarchitektur
[[1.000]
 [0.000]
 [0.000]]
-----------v-----------
[[  0   0   0]
 [-10  20  20]
 [ 30 -20 -20]]
-----------v-----------
[[1.000 1.000 1.000]
 [0.000 0.000 0.000]
 [0.000 0.000 0.000]]
-----------v-----------
[[  0   0   0]
 [-30  20  20]]
-----------v-----------
[[0.000 0.000 0.000]
 [0.000 0.000 0.000]]
-----------v-----------
Predict:
[1 1 1] 0 -> [0.000]
[1 0 1] 1 -> [1.000]
[1 1 0] 1 -> [1.000]
[1 0 0] 0 -> [0.000]
